In [1]:
import os

In [2]:
list_file = list(os.walk('./'))[0][2]
for file in list_file:
    if '替代料总表' in file:
        alternate = './' + file

print(f'\talternate: \t{alternate}')

	alternate: 	./替代料总表08-14.xlsx


In [3]:
import pandas as pd

In [4]:
df_alternate = pd.read_excel(alternate,
                             header=1)[['主料SKU', '替换料1', '替换料2', '替换料3', '替换料4', '替换料5', '替换料6']].fillna('')

df_alternate = df_alternate[df_alternate['替换料1'] != ''].reset_index(drop=True)

df_alternate

,主料SKU,替换料1,替换料2,替换料3,替换料4,替换料5,替换料6
0,USCA171876D-3,USCA171876D,USCA171876-3,USCA171876,,,
1,USWM42150B,USWM42150,,,,,
2,USGS37022T,USGS37022G,USGS37022,USGS37022J,,,
3,USGS36761TB,USGS36761,USGS36761T,,,,
4,USGS41570B,USGS41570,,,,,
...,...,...,...,...,...,...,...
34452,USHHS41307-C,USHHS41307,,,,,
34453,USHHS41308-C,USHHS41308,,,,,
34454,USHHS94130-C,USHHS94130,,,,,
34455,USHHS94131-C,USHHS94131,,,,,


In [5]:
from tqdm import tqdm

In [6]:
count = 0
while True:
    print('# = = = = = = = ' + str(count) + ' = = = = = = = = = =')
    list_row = []
    for i in tqdm(range(len(df_alternate)), desc='运行中', ncols=77):
        list_sku = list(set(df_alternate.loc[i].tolist()))
        if '' in list_sku:
            list_sku.remove('')

        find = False
        for sku in list_sku:
            for j in range(len(list_row)):
                if sku in list_row[j]:
                    list_row[j] = list(set(list_row[j] + list_sku))
                    find = True
                    break
            if find == True:
                break

        if find == False:
            list_row.append(list_sku)  
        
    if len(list_row) == len(df_alternate):
        break
    else:
        df_alternate = pd.DataFrame(list_row).fillna('')
        df_alternate.columns = ['主料SKU'] + ['替换料'+str(i) for i in range(1, len(df_alternate.columns))]
        count += 1
        print()

# = = = = = = = 0 = = = = = = = = = =


运行中: 100%|█████████████████████████| 34457/34457 [02:09<00:00, 267.10it/s]



# = = = = = = = 1 = = = = = = = = = =


运行中: 100%|█████████████████████████| 34445/34445 [02:09<00:00, 265.10it/s]


In [7]:
import datetime

In [8]:
writer = pd.ExcelWriter('Lennon_整理好的替代料总表_' + datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + '.xlsx')
df_alternate.to_excel(writer, index=False)
writer.close()